In [90]:
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random
from jax import jacfwd, jacrev
import numpy as np
from numpy.linalg import inv

dyn_pendulum = {"g":9.81, "m": 1.0, "l": 1.0}
x0 = np.array([0.1, 0.0])
v1 = np.array([1.0, 0.0])
v2 = np.array([0.0, 1.0])
x_dot = []
x_norm = []

def simple_pendulum (x0, dyn):
    g = dyn['g'] 
    l = dyn['l']
    #print (x0)
    
    return np.array([x0[1], (-g/l)*np.sin(x0[0])])

def simple_pendulum_jax (x0, dyn):
    g = dyn['g'] 
    l = dyn['l']
    return jnp.array([x0[1], (-g/l)*jnp.sin(x0[0])])

def linearized_simple_pendulum (x0, dyn, y_pendulum):
    g = dyn['g'] 
    l = dyn['l']
    #print (x0)
    return np.array(jnp.dot(jacfwd(simple_pendulum_jax)(x0,dyn),y_pendulum))

# def linearized_simple_pendulum (x0, dyn, y_pendulum):
#     g = dyn['g'] 
#     l = dyn['l']
#     #print (x0)
#     return np.array([0*x0[0] + (-g/l)*np.cos(y_pendulum[0])*x0[1],
#                      1*x0[0] + 0*x0[1]])

# def RungeKutta_for_collect_trajectory (dyn, f, dt, x0):
#     k1 = f(x0, dyn) #[x,y,z]*0.1 example
#     k2 = f(x0+0.5*k1*dt,dyn)
#     k3 = f(x0 + 0.5*k2*dt, dyn)
#     k4 = f(x0 + k3*dt, dyn)
    
#     x = x0 + ((k1 + 2*k2 + 2*k3 + k4)/6) *dt
#     x_norm.append(x0)
#     x_dot.append(k4)
#     return x

def RungeKutta (dyn, f, dt, x0):
    k1 = f(x0, dyn) #[x,y,z]*0.1 example
    k2 = f(x0+0.5*k1*dt,dyn)
    k3 = f(x0 + 0.5*k2*dt, dyn)
    k4 = f(x0 + k3*dt, dyn)
    
    x = x0 + ((k1 + 2*k2 + 2*k3 + k4)/6) *dt
    return x

def RungeKutta_linearized (dyn, f, dt, x0, y):
    k1 = f(x0, dyn, y) #[x,y,z]*0.1 example
    k2 = f(x0+0.5*k1*dt,dyn, y)
    k3 = f(x0 + 0.5*k2*dt, dyn, y)
    k4 = f(x0 + k3*dt, dyn, y)
    
    x = x0 + ((k1 + 2*k2 + 2*k3 + k4)/6) * dt

    return x

def f_t (dyn, f, linearized_f, dt, x0, T):
    x = np.empty(shape=(len(x0),T))
    v1_prime = np.empty(shape=(len(x0),T))
    v2_prime = np.empty(shape=(len(x0),T))
    x[:, 0] = x0
    v1_prime[:, 0] = v1
    v2_prime[:, 0] = v2
    cum = np.array([0,0])
    
    for i in range(1,T):
        x[:, i] = RungeKutta(dyn, f, dt, x[:, i-1])
        
        v1_prime[:, i] = RungeKutta_linearized(dyn, linearized_f, dt, v1_prime[:, i-1], x[:, i-1])
        v2_prime[:, i] = RungeKutta_linearized(dyn, linearized_f, dt, v2_prime[:, i-1], x[:, i-1])
        
#         v_prime = np.array([v1_prime[:, i],v2_prime[:, i]])
        
#         v_prime_new = gram_schmidt(v_prime)
#         norm = np.linalg.norm(v_prime_new)
        norm1 = np.linalg.norm(v1_prime[:, i])
        v1_prime[:, i] = v1_prime[:, i]/norm1
        
        GSC1 = np.dot(v1_prime[:, i], v2_prime[:, i])
        v2_prime[:, i] = v2_prime[:, i] - GSC1*v1_prime[:, i]
        
        norm2 = np.linalg.norm(v2_prime[:, i])
        v2_prime[:, i] = v2_prime[:, i]/norm2
        

        cum = cum + np.log2(np.array([norm1,norm2]))
#         print("total time taken this loop: ", end_time - start_time)
        
    return x, cum/(T*dt)

In [91]:
_, cum = f_t(dyn_pendulum, simple_pendulum, linearized_simple_pendulum, 0.001, np.array([0,0.1]), 500)

In [92]:
cum

array([ 0.09635589, -0.28452818])